In [2]:

import cv2
#from google.colab.patches import cv2_imshow
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from statistics import stdev,mean


#define globals required through out the whole program
edges=[] #containing all edge tuple
attrs=[] #countaining list of attribute of all nodes
graph_id=1 #id of latest graph
node_id=1 #id of latest node
graph_indicator=[] #containing graph-id for each node
node_labels=[] #containing labels for all node
graph_labels=[]#containing labels for all graph

#activity-label vs activity-name mapping (4-class)
activity_map={}
activity_map[1]='glioma'

activity_map[2]='meningioma'
activity_map[3]='pituitary'

#z-score normalization
def normalize(arr):
    arr=np.array(arr)
    m=np.mean(arr)
    s=np.std(arr)
    return (arr-m)/s

#generate graph for a given edge-image file
def generate_graphs(filename,node_label,activity_map):
    print(" ... Reading image: "+filename+" ...")
    global node_id,edges,attrs,graph_id,node_labels,graph_indicator
    cnt=0
    img=cv2.imread(filename)
    dim1,dim2,_=img.shape
    attrs1=[]

    print("Image type: " + activity_map[node_label] + "\nPixel matrix is of: "+str(dim1)+"x"+str(dim2))
    img1=img.copy()
    nodes=np.full((dim1,dim2),-1)
    edge=0
    for i in range(dim1):
        for j in range(dim2):
      #considering pixel as node if pixel-value>=128
            b,_,_=img[i][j]
            if(b>=128):
                nodes[i][j]=node_id
                attrs1.append(b)
                graph_indicator.append(graph_id)
                node_labels.append([node_label,activity_map[node_label]])
                node_id+=1
                cnt+=1
            else:
                img1[i][j]=0
    for i in range(dim1):
        for j in range(dim2):
      #forming edge between all adjacent pixels which are node
          if(nodes[i][j]!=-1):
            li=max(0,i-1)
            ri=min(i+2,dim1)
            lj=max(0,j-1)
            rj=min(j+2,dim2)
            for i1 in range(li,ri):
                for j1 in range(lj,rj):
                    if((i1!=i or j1!=j) and (nodes[i1][j1]!=-1)):
                        edges.append([nodes[i][j],nodes[i1][j1]])
                        edge+=1
    attrs1=normalize(attrs1)
    attrs.extend(attrs1)
    del attrs1
    print("For given image nodes formed: "+str(cnt)+" edges formed: "+str(edge))
    if(cnt!=0): graph_id+=1

#generate graphs for all edge-image under given dir along with proper label
def generate_graph_with_labels(dirname,label,activity_map):
    print("\n... Reading Directory: "+dirname+" ...\n")
    global graph_labels
    filenames=glob.glob(dirname+'/*.png')
    for filename in filenames:
        generate_graphs(filename,label,activity_map)
        graph_labels.append([label,activity_map[label]])

#generate graphs for all directories
def process_graphs(covid_dir,bac_dir,vir_dir,activity_map):
    global node_labels,graph_labels
    generate_graph_with_labels(covid_dir,1,activity_map)
    #generate_graph_with_labels(norm_dir,2,activity_map)
    generate_graph_with_labels(bac_dir,2,activity_map)
    generate_graph_with_labels(vir_dir,3,activity_map)
    print("Processing done")
    print("Total nodes formed: "+str(len(node_labels))+"Total graphs formed: "+str(len(graph_labels)))

#working directories
covid_dir='brain3/Pretwitt/glioma' #for covid
#norm_dir='brain4/Pretwitt/notumor' #for normal
bac_dir='brain3/Pretwitt/meningioma' #for bacterial-pneumonia
vir_dir='brain3/Pretwitt/pituitary' #for viral-pneumonia

process_graphs(covid_dir,bac_dir,vir_dir,activity_map)

#check all the lengths of globals
#comment if not necessary
print(len(node_labels))
print(len(graph_labels))
print(len(edges))
print(len(attrs))

#create adjacency dataframe
df_A=pd.DataFrame(columns=["node-1","node-2"],data=np.array(edges))
print("Shape of edge dataframe: "+str(df_A.shape))
print("\n--summary of dataframe--\n" ,df_A.head(50))

#create node label dataframe
df_node_label=pd.DataFrame(data=np.array(node_labels),columns=["label","activity-name"])
print("shape of node-label dataframe: "+str(df_node_label.shape))
print("\n--summary of dataframe--\n" ,df_node_label)

#create graph label dataframe
df_graph_label=pd.DataFrame(data=np.array(graph_labels),columns=["label","activity-name"])
print("shape of node-label dataframe: "+str(df_graph_label.shape))
print("\n--summary of dataframe--\n" ,df_graph_label.head(50))

#create node-attribute dataframe (normalized grayscale value)
df_node_attr=pd.DataFrame(data=np.array(attrs),columns=["gray-val"])
print("shape of node-attribute dataframe: "+str(df_node_attr.shape))
print("\n--summary of dataframe--\n" ,df_node_attr.head(50))

#create graph-indicator datframe
df_graph_indicator=pd.DataFrame(data=np.array(graph_indicator),columns=["graph-id"])
print("shape of graph-indicator dataframe: "+str(df_graph_indicator.shape))
print("\n--summary of dataframe--\n" ,df_graph_indicator.head(50))

#omit activity name later for graph-label and node-label
#since GIN model will only accept the label
df_node_label=df_node_label.drop(["activity-name"],axis=1)
print(df_node_label.head(50))

df_graph_label=df_graph_label.drop(["activity-name"],axis=1)
print(df_graph_label.head(50))



def save_dataframe_to_txt(df,filepath):
    df.to_csv(filepath,header=None,index=None,sep=',',mode='w')



#save all the dataframes to .txt file
#path name: .../GraphTrain/dataset/<dataset_name>/raw/<dataset_name>_<type>.txt
# <type>:
# A--> adjancency matrix
#graph_indicator--> graph-ids of all node
#graph_labels--> labels for all graph
#node_attributes--> attribute(s) for all node
#node_labels--> labels for all node

sourcepath='brain3/Github_Pretwitt/raw'
save_dataframe_to_txt(df_A,sourcepath+'/Github_3class_Pretwitt_A.txt')
save_dataframe_to_txt(df_graph_indicator,sourcepath+'/Github_3class_Pretwitt_graph_indicator.txt')
save_dataframe_to_txt(df_graph_label,sourcepath+'/Github_3class_Pretwitt_graph_labels.txt')
save_dataframe_to_txt(df_node_attr,sourcepath+'/Github_3class_Pretwitt_node_attributes.txt')
save_dataframe_to_txt(df_node_label,sourcepath+'/Github_3class_Pretwitt_node_labels.txt')


... Reading Directory: brain3/Pretwitt/glioma ...

 ... Reading image: brain3/Pretwitt/glioma\img-1.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 2121 edges formed: 7516
 ... Reading image: brain3/Pretwitt/glioma\img-10.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 1815 edges formed: 6758
 ... Reading image: brain3/Pretwitt/glioma\img-100.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 108 edges formed: 258
 ... Reading image: brain3/Pretwitt/glioma\img-101.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 1826 edges formed: 7664
 ... Reading image: brain3/Pretwitt/glioma\img-102.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 1723 edges formed: 7214
 ... Reading image: brain3/Pretwitt/glioma\img-103.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 246 edges formed:

For given image nodes formed: 637 edges formed: 2514
 ... Reading image: brain3/Pretwitt/glioma\img-146.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 2285 edges formed: 8888
 ... Reading image: brain3/Pretwitt/glioma\img-147.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 113 edges formed: 408
 ... Reading image: brain3/Pretwitt/glioma\img-148.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 1239 edges formed: 4334
 ... Reading image: brain3/Pretwitt/glioma\img-149.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 336 edges formed: 1188
 ... Reading image: brain3/Pretwitt/glioma\img-15.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 968 edges formed: 3964
 ... Reading image: brain3/Pretwitt/glioma\img-150.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 502 edges formed

For given image nodes formed: 997 edges formed: 4914
 ... Reading image: brain3/Pretwitt/glioma\img-193.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 677 edges formed: 2560
 ... Reading image: brain3/Pretwitt/glioma\img-194.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 1640 edges formed: 6438
 ... Reading image: brain3/Pretwitt/glioma\img-195.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 1614 edges formed: 8002
 ... Reading image: brain3/Pretwitt/glioma\img-196.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 3612 edges formed: 16484
 ... Reading image: brain3/Pretwitt/glioma\img-197.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 4077 edges formed: 18516
 ... Reading image: brain3/Pretwitt/glioma\img-198.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 2792 edges

For given image nodes formed: 2027 edges formed: 8036
 ... Reading image: brain3/Pretwitt/glioma\img-40.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 81 edges formed: 314
 ... Reading image: brain3/Pretwitt/glioma\img-41.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 2780 edges formed: 12952
 ... Reading image: brain3/Pretwitt/glioma\img-42.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 630 edges formed: 2182
 ... Reading image: brain3/Pretwitt/glioma\img-43.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 540 edges formed: 2030
 ... Reading image: brain3/Pretwitt/glioma\img-44.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 1108 edges formed: 3368
 ... Reading image: brain3/Pretwitt/glioma\img-45.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 1157 edges formed: 5

For given image nodes formed: 1217 edges formed: 5172
 ... Reading image: brain3/Pretwitt/glioma\img-88.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 3051 edges formed: 12290
 ... Reading image: brain3/Pretwitt/glioma\img-89.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 1703 edges formed: 7604
 ... Reading image: brain3/Pretwitt/glioma\img-9.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 1210 edges formed: 4982
 ... Reading image: brain3/Pretwitt/glioma\img-90.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 3014 edges formed: 13470
 ... Reading image: brain3/Pretwitt/glioma\img-91.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 2352 edges formed: 8680
 ... Reading image: brain3/Pretwitt/glioma\img-92.png ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 632 edges forme

...

For given image nodes formed: 930 edges formed: 3926
 ... Reading image: brain3/Pretwitt/meningioma\img-86.png ...
Image type: meningioma
Pixel matrix is of: 512x512
For given image nodes formed: 1353 edges formed: 5324
 ... Reading image: brain3/Pretwitt/meningioma\img-87.png ...
Image type: meningioma
Pixel matrix is of: 512x512
For given image nodes formed: 899 edges formed: 3616
 ... Reading image: brain3/Pretwitt/meningioma\img-88.png ...
Image type: meningioma
Pixel matrix is of: 512x512
For given image nodes formed: 1365 edges formed: 4592
 ... Reading image: brain3/Pretwitt/meningioma\img-89.png ...
Image type: meningioma
Pixel matrix is of: 512x512
For given image nodes formed: 1568 edges formed: 7058
 ... Reading image: brain3/Pretwitt/meningioma\img-9.png ...
Image type: meningioma
Pixel matrix is of: 512x512
For given image nodes formed: 339 edges formed: 1280
 ... Reading image: brain3/Pretwitt/meningioma\img-90.png ...
Image type: meningioma
Pixel matrix is of: 512x512
Fo

For given image nodes formed: 3648 edges formed: 17294
 ... Reading image: brain3/Pretwitt/pituitary\img-13.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 2500 edges formed: 9756
 ... Reading image: brain3/Pretwitt/pituitary\img-130.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 4034 edges formed: 20212
 ... Reading image: brain3/Pretwitt/pituitary\img-131.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 2484 edges formed: 11414
 ... Reading image: brain3/Pretwitt/pituitary\img-132.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 4419 edges formed: 22812
 ... Reading image: brain3/Pretwitt/pituitary\img-133.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 2712 edges formed: 13882
 ... Reading image: brain3/Pretwitt/pituitary\img-134.png ...
Image type: pituitary
Pixel matrix is of: 512x512


For given image nodes formed: 2762 edges formed: 12734
 ... Reading image: brain3/Pretwitt/pituitary\img-175.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 2564 edges formed: 11662
 ... Reading image: brain3/Pretwitt/pituitary\img-176.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 3046 edges formed: 14206
 ... Reading image: brain3/Pretwitt/pituitary\img-177.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 669 edges formed: 2952
 ... Reading image: brain3/Pretwitt/pituitary\img-178.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 3078 edges formed: 14852
 ... Reading image: brain3/Pretwitt/pituitary\img-179.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 704 edges formed: 2862
 ... Reading image: brain3/Pretwitt/pituitary\img-18.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For

For given image nodes formed: 2295 edges formed: 11654
 ... Reading image: brain3/Pretwitt/pituitary\img-22.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 2238 edges formed: 9930
 ... Reading image: brain3/Pretwitt/pituitary\img-220.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 3666 edges formed: 19492
 ... Reading image: brain3/Pretwitt/pituitary\img-221.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 2620 edges formed: 13516
 ... Reading image: brain3/Pretwitt/pituitary\img-222.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 2693 edges formed: 13830
 ... Reading image: brain3/Pretwitt/pituitary\img-223.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 2409 edges formed: 11450
 ... Reading image: brain3/Pretwitt/pituitary\img-224.png ...
Image type: pituitary
Pixel matrix is of: 512x512


For given image nodes formed: 2688 edges formed: 13734
 ... Reading image: brain3/Pretwitt/pituitary\img-60.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 1332 edges formed: 5860
 ... Reading image: brain3/Pretwitt/pituitary\img-61.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 1262 edges formed: 6000
 ... Reading image: brain3/Pretwitt/pituitary\img-62.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 2171 edges formed: 9918
 ... Reading image: brain3/Pretwitt/pituitary\img-63.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 2524 edges formed: 11090
 ... Reading image: brain3/Pretwitt/pituitary\img-64.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 2820 edges formed: 13600
 ... Reading image: brain3/Pretwitt/pituitary\img-65.png ...
Image type: pituitary
Pixel matrix is of: 512x512
For giv